In [ ]:
import pandas as pd

ML_AG_1 = pd.read_csv("auto_submission_feature.csv")
ML_AG_2 = pd.read_csv("auto_submission_feature2.csv")
ML_AG_4 = pd.read_csv("auto_submission_feature4.csv")
ML_AG_6 = pd.read_csv("auto_submission_feature6.csv")
ML_AG_7 = pd.read_csv("auto_submission_feature7.csv")
ML_AG_8 = pd.read_csv("auto_submission.csv")
ML_AG_1.loc[ML_AG_1['answer'] < 0.0, 'answer'] = 0.0
ML_AG_2.loc[ML_AG_2['answer'] < 0.0, 'answer'] = 0.0
ML_AG_4.loc[ML_AG_4['answer'] < 0.0, 'answer'] = 0.0
ML_AG_6.loc[ML_AG_6['answer'] < 0.0, 'answer'] = 0.0
ML_AG_7.loc[ML_AG_7['answer'] < 0.0, 'answer'] = 0.0
ML_AG_8.loc[ML_AG_8['answer'] < 0.0, 'answer'] = 0.0

ML_AG = ML_AG_1.copy()
ML_AG['answer'] = (ML_AG_1['answer'] + ML_AG_2['answer'] + ML_AG_4['answer'] + ML_AG_6['answer'] + ML_AG_7['answer'] + ML_AG_8['answer']) / 6

ML_AG_1_v2 = pd.read_csv("autogluon_regression_real_final.csv")
ML_AG_2_v2 = pd.read_csv("autogluon_regression_final222.csv")
ML_AG_3_v2 = pd.read_csv("autogluon_regression_real_final_WeightedEnsemble_L3.csv")
ML_AG_1_v2.loc[ML_AG_1_v2['answer'] < 0.0, 'answer'] = 0.0
ML_AG_2_v2.loc[ML_AG_2_v2['answer'] < 0.0, 'answer'] = 0.0
ML_AG_3_v2.loc[ML_AG_3_v2['answer'] < 0.0, 'answer'] = 0.0
ML_AG_v2 = ML_AG_1_v2.copy()
ML_AG_v2['answer'] = (ML_AG_1_v2['answer'] + ML_AG_2_v2['answer'] + ML_AG_3_v2['answer']) / 3


XGB_1 = pd.read_csv("xgboost_timeseries.csv")
XGB_2 = pd.read_csv("xgboost_timeseries2.csv")
XGB_1.loc[XGB_1['answer'] < 0.0, 'answer'] = 0.0
XGB_2.loc[XGB_2['answer'] < 0.0, 'answer'] = 0.0

XGB = XGB_1.copy()
XGB['answer'] = (XGB_1['answer'] + XGB_2['answer']) / 2

mlj_1 = pd.read_csv("MJSEVSEVSEV.csv")
mlj_2 = pd.read_csv("MLJAR_21011928.csv")
mlj_3 = pd.read_csv("MLJAR_19990313_NEW1_REAL.csv")
mlj_1.loc[mlj_1['answer'] < 0.0, 'answer'] = 0.0
mlj_2.loc[mlj_2['answer'] < 0.0, 'answer'] = 0.0
mlj_3.loc[mlj_3['answer'] < 0.0, 'answer'] = 0.0


mlj = mlj_1.copy()
mlj['answer'] = (mlj_1['answer'] + mlj_2['answer'] + mlj_3['answer']) / 3


DL_1 = pd.read_csv("TG_TOALL_RE1.csv")
DL_2 = pd.read_csv("TG_TOALL_RE1_seed.csv") 
DL_1.loc[DL_1['answer'] < 0.0, 'answer'] = 0.0
DL_2.loc[DL_2['answer'] < 0.0, 'answer'] = 0.0

DL = DL_1.copy()
DL['answer'] = (DL_1['answer']*0.2) + (DL_2['answer']*0.8)


Final = DL_1.copy()


Final['date'] = Final['ID'].str.extract(r'_(\d{8})$')[0]
Final['date'] = pd.to_datetime(Final['date'], format='%Y%m%d')
date_threshold = pd.Timestamp('2023-03-19')
before_threshold = Final['date'] <= date_threshold
after_threshold = Final['date'] > date_threshold
Final['item'] = Final['ID'].str.extract(r'^([A-Za-z]+)_')[0]
is_TG = Final['item'] == 'TG'
not_TG = ~is_TG
Final.loc[before_threshold, 'answer'] = (
    DL.loc[before_threshold, 'answer'] * 0.6 +
    ML_AG.loc[before_threshold, 'answer'] * 0.1 +
    ML_AG_v2.loc[before_threshold, 'answer'] * 0.1 +
    XGB.loc[before_threshold, 'answer'] * 0.1 +
    mlj.loc[before_threshold, 'answer'] * 0.1
)
Final.loc[after_threshold & is_TG, 'answer'] = (
    XGB.loc[after_threshold & is_TG, 'answer'] * 0.3 +
    mlj.loc[after_threshold & is_TG, 'answer'] * 0.4 + 
    ML_AG_v2.loc[after_threshold, 'answer'] * 0.3
)
Final.loc[after_threshold & not_TG, 'answer'] = (
    DL.loc[after_threshold, 'answer'] * 0.6 +
    ML_AG.loc[after_threshold, 'answer'] * 0.1 +
    ML_AG_v2.loc[after_threshold, 'answer'] * 0.1 +
    XGB.loc[after_threshold, 'answer'] * 0.1 +
    mlj.loc[after_threshold, 'answer'] * 0.1
)

Final.drop(columns=['date'], inplace=True)

Final['itemcorloc'] = Final["ID"].str[:6]
Final["answer"][Final["itemcorloc"] == "RD_E_S"] = Final["answer"][Final["itemcorloc"] == "RD_E_S"]*2
Final["answer"][Final["itemcorloc"] == "RD_E_S"]
Final.drop(['itemcorloc'],axis=1,inplace=True)



train = pd.read_csv("train.csv")
dates_to_update = pd.date_range(start='2023-03-04', end='2023-03-12').strftime('%Y-%m-%d').tolist()
filtered_data = train[(train['timestamp'].isin(['2023-03-01', '2023-03-02', '2023-03-03'])) & (train['supply(kg)'] == 0)]
grouped_data = filtered_data.groupby(['item', 'corporation', 'location']).size().reset_index(name='count')
combinations_with_zero_supply = grouped_data[grouped_data['count'] == 3]
for index, row in combinations_with_zero_supply.iterrows():
    item, corporation, location = row['item'], row['corporation'], row['location']
    for date in dates_to_update:
        formatted_date = date.replace('-', '')
        id_value = f"{item}_{corporation}_{location}_{formatted_date}"
        Final.loc[Final['ID'] == id_value, 'answer'] = 0
dates_to_filter = pd.date_range(start='2023-02-21', end='2023-03-03').strftime('%Y-%m-%d').tolist()
dates_to_update = pd.date_range(start='2023-03-04', end='2023-03-31').strftime('%Y-%m-%d').tolist()
filtered_data = train[(train['timestamp'].isin(dates_to_filter)) & (train['supply(kg)'] == 0)]
grouped_data = filtered_data.groupby(['item', 'corporation', 'location']).size().reset_index(name='count')
days_in_filter_range = len(dates_to_filter)
combinations_with_zero_supply = grouped_data[(grouped_data['count'] == days_in_filter_range) & ((grouped_data['item'] == 'BC') | (grouped_data['item'] == 'RD') | (grouped_data['item'] == 'CR'))]

final_data = Final.copy()
for index, row in combinations_with_zero_supply.iterrows():
    item, corporation, location = row['item'], row['corporation'], row['location']
    for date in dates_to_update:
        formatted_date = date.replace('-', '')
        id_value = f"{item}_{corporation}_{location}_{formatted_date}"
        final_data.loc[final_data['ID'] == id_value, 'answer'] = 0

Final = final_data.copy()
sunday_ids_march_2023 = [f"_202303{day:02}" for day in [5, 12, 19, 26]]
for sunday_id in sunday_ids_march_2023:
    Final.loc[Final['ID'].str.contains(sunday_id), 'answer'] = 0
Final.drop(['item'],axis=1,inplace=True)
Final.to_csv('FINAL_2.csv',index=False)